In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, models, transforms

import numpy as np
from PIL import Image

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
data_transforms = transforms.Compose([
        transforms.CenterCrop((384, 512)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])

In [3]:
from inceptionresnetv2 import inceptionresnetv2
class model_qa(nn.Module):
    def __init__(self,num_classes,**kwargs):
        super(model_qa,self).__init__()
        base_model = inceptionresnetv2(num_classes=1000, pretrained='imagenet')
        self.base= nn.Sequential(*list(base_model.children())[:-1])
        self.fc = nn.Sequential(
            nn.Linear(1536, 2048),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(2048),
            nn.Dropout(p=0.25),
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(1024),
            nn.Dropout(p=0.25),
            nn.Linear(1024, 256),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(256),         
            nn.Dropout(p=0.5),
            nn.Linear(256, num_classes),
        )

    def forward(self,x):
        x = self.base(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)

        return x

In [4]:
'''
Please follow the instructions in the GitHub page to download the pre-trained model:
https://github.com/multimediaeval/2020-Pixel-Privacy-Task
'''

KonCept512 = model_qa(num_classes=1) 
KonCept512.load_state_dict(torch.load('./KonCept512.pth'))
KonCept512.eval().to(device)

model_qa(
  (base): Sequential(
    (0): BasicConv2d(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (1): BasicConv2d(
      (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (2): BasicConv2d(
      (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
    (3): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): BasicConv2d(
      (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
    )
 

### Before conversion

In [5]:
img_paths = ['../pp2020_dev/11396447303.jpg', '../pp2020_dev/107505789.jpg', '../pp2020_dev/10692667386.jpg']
imgs = [data_transforms(Image.open(img_path).convert('RGB')) for img_path in img_paths]
score = KonCept512(torch.stack(imgs).to(device)).detach().cpu().numpy().squeeze()
for i, img_path in enumerate(img_paths):
    print('The BIQA score of image `{}` is {}'.format(img_path, score[i]))

The BIQA score of image `../pp2020_dev/11396447303.jpg` is 67.12537384033203
The BIQA score of image `../pp2020_dev/107505789.jpg` is 57.88755798339844
The BIQA score of image `../pp2020_dev/10692667386.jpg` is 65.55703735351562


### Conversion to TF via ONNX

In [6]:
onnx_model_name = "biqa.onnx"
input_names = ["input_image"]
output_names = ["biqa_score"]
dynamic_axes = {input_names[0] : {0: "batch_size"}, output_names[0] : {0: "batch_size"}}
input_shape = [1, 3, 384, 512]
dummy_input = torch.randn(*input_shape, device='cuda')
torch.onnx.export(KonCept512, dummy_input, onnx_model_name, verbose=True, input_names=input_names, output_names=output_names, dynamic_axes=dynamic_axes)

graph(%input_image : Float(1:589824, 3:196608, 384:512, 512:1, requires_grad=0, device=cuda:0),
      %base.8.0.conv2d.weight : Float(320:128, 128:1, 1:1, 1:1, requires_grad=1, device=cuda:0),
      %base.8.0.conv2d.bias : Float(320:1, requires_grad=1, device=cuda:0),
      %base.8.1.conv2d.weight : Float(320:128, 128:1, 1:1, 1:1, requires_grad=1, device=cuda:0),
      %base.8.1.conv2d.bias : Float(320:1, requires_grad=1, device=cuda:0),
      %base.8.2.conv2d.weight : Float(320:128, 128:1, 1:1, 1:1, requires_grad=1, device=cuda:0),
      %base.8.2.conv2d.bias : Float(320:1, requires_grad=1, device=cuda:0),
      %base.8.3.conv2d.weight : Float(320:128, 128:1, 1:1, 1:1, requires_grad=1, device=cuda:0),
      %base.8.3.conv2d.bias : Float(320:1, requires_grad=1, device=cuda:0),
      %base.8.4.conv2d.weight : Float(320:128, 128:1, 1:1, 1:1, requires_grad=1, device=cuda:0),
      %base.8.4.conv2d.bias : Float(320:1, requires_grad=1, device=cuda:0),
      %base.8.5.conv2d.weight : Float(3

In [7]:
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load(onnx_model_name)  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation

/home/mathsci/miniconda3/envs/n-cript/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:38: UserWarning: You are currently using a nightly version of TensorFlow (2.4.0-dev20201023). 
TensorFlow Addons offers no support for the nightly versions of TensorFlow. Some things might work, some other might not. 
If you encounter a bug, do not file an issue on GitHub.
  warnings.warn(


### After Conversion

In [8]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import CenterCrop, Normalization

center_crop = CenterCrop(384, 512)
normalization = Normalization(axis=1)
normalization.build(input_shape)
normalization._set_state_variables({"mean": [0.5, 0.5, 0.5], "variance": [0.5**2, 0.5**2, 0.5**2]})

def preprocess(image_path):
    image_raw = tf.io.read_file(image_path)
    image = tf.image.decode_image(image_raw)
    image = tf.cast(image, tf.float32)
    image /= 255.0
    image = image[None, ...]
    image = center_crop(image)
    image = tf.transpose(image, perm=[0, 3, 1, 2])
    image = normalization(image)
    return image

# tf_model = tf.saved_model.load(tf_model_name)
tf_model = tf_rep.tf_module
imgs = [preprocess(img_path) for img_path in img_paths]
score = tf.squeeze(tf_model(input_image=tf.concat(imgs, axis=0))[0])
for i, img_path in enumerate(img_paths):
    print('The BIQA score of image `{}` is {}'.format(img_path, score[i]))

The BIQA score of image `../pp2020_dev/11396447303.jpg` is 67.04446411132812
The BIQA score of image `../pp2020_dev/107505789.jpg` is 58.35835266113281
The BIQA score of image `../pp2020_dev/10692667386.jpg` is 65.37245178222656
